<a href="https://colab.research.google.com/github/joooser/TENSORFLOW2/blob/main/json_tfrecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import pickle
import zipfile
!pip install tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 4.3 MB/s 


In [7]:
import pandas as pd

In [2]:
local_zip = "/content/carros.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("carros_original")
zip_ref.close()

In [5]:
type_file = "train"
path = "/content/train.json"
data = json.load(open(path))

In [6]:
data

[{'classification': [],
  'height': '953',
  'id': '4e59f0ea-c47d-42c1-a201-b3f158c0a93a7ab94317-b86c-45bd-924b-b2016321f41d',
  'image': 'fc15988c9a.jpg',
  'tags': [{'classes': [],
    'color': '#f10c27',
    'id': 'd3de93b5-309a-45d2-b072-542118fe66c5',
    'name': 'motos',
    'parent': None,
    'pos': {'h': 427.7056277056277,
     'w': 746.2337662337662,
     'x': 63.03030303030303,
     'y': 436.7099567099567},
    'type': 'bounding_box'}],
  'width': '1300'},
 {'classification': [],
  'height': '635',
  'id': '4e59f0ea-c47d-42c1-a201-b3f158c0a93a95c26985-cde0-4a1a-b499-bdd9be8b70f0',
  'image': '0c535fcdfa.jpg',
  'tags': [{'classes': [],
    'color': '#f10c27',
    'id': '7876b769-5390-4607-89da-ea363c5fd73d',
    'name': 'motos',
    'parent': None,
    'pos': {'h': 203, 'w': 146, 'x': 578, 'y': 237},
    'type': 'bounding_box'},
   {'classes': [],
    'color': '#f10c27',
    'id': 'ce0ba9b0-09d4-4734-974b-54f9e99feddb',
    'name': 'motos',
    'parent': None,
    'pos': {'h

In [10]:
csv_list = []

for classification in data:
  width, height = classification["width"], classification["height"]
  image = classification["image"]
  for item in classification["tags"]:
    name = item["name"]
    xmin = item["pos"]["x"]
    ymin = item["pos"]["y"]
    xmax = item["pos"]["x"] + item["pos"]["w"]
    ymax = item["pos"]["y"] + item["pos"]["h"]
  
    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    csv_list.append(value)
  
  column_name = ["filename", "width", "height", "class", "xmin", "ymin", "xmax", "ymax"]
  csv_df = pd.DataFrame(csv_list, columns = column_name)

  csv_df.to_csv("/content/{}_labels.csv".format(type_file))


In [11]:
type_file = "test"
path = "/content/test.json"
data = json.load(open(path))

In [12]:
csv_list = []

for classification in data:
  width, height = classification["width"], classification["height"]
  image = classification["image"]
  for item in classification["tags"]:
    name = item["name"]
    xmin = item["pos"]["x"]
    ymin = item["pos"]["y"]
    xmax = item["pos"]["x"] + item["pos"]["w"]
    ymax = item["pos"]["y"] + item["pos"]["h"]
  
    value = (image, width, height, name, xmin, ymin, xmax, ymax)
    csv_list.append(value)
  
  column_name = ["filename", "width", "height", "class", "xmin", "ymin", "xmax", "ymax"]
  csv_df = pd.DataFrame(csv_list, columns = column_name)

  csv_df.to_csv("/content/{}_labels.csv".format(type_file))

In [13]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Note: checking out '58d19c67e1d30d905dd5c6e5092348658fed80af'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 58d19c67e Internal change
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 h

In [20]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf
import sys
sys.path.append("../../models/research")

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

def class_text_to_int(row_label):
    if row_label == "carro":
        return 1
    elif row_label == "moto":
        return 2
    else:
        None


def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

output_path = "train.record"
img_path = "/content/carros_original/dataset_original/"
csv_input = "/content/train_labels.csv"

writer = tf.io.TFRecordWriter(output_path)
path = os.path.join(img_path)
examples = pd.read_csv(csv_input)
grouped = split(examples, 'filename')
for group in grouped:
    tf_example = create_tf_example(group, path)
    writer.write(tf_example.SerializeToString())

writer.close()
output_path = os.path.join(os.getcwd(), output_path)
print('Successfully created the TFRecords: {}'.format(output_path))




TypeError: ignored